
# Spatiotemporal Precipitation Prediction
**5 × 5 Experiments Notebook**  
Train & validate five architectures across five temporal folds (48 m train → 12 m val).  Designed to run **locally or on Google Colab** — auto‑detects GPU/CPU and adapts parallelism.

In [ ]:

# ▶️ Prevent kernel crashes due to CDN and memory issues
import os
import sys
import warnings

# 1. Disable CDN access to prevent widget errors
os.environ['JUPYTER_DISABLE_MATHJAX'] = '1'  # Disable MathJax (uses CDN)
os.environ['TQDM_DISABLE'] = '1'  # Avoid tqdm widgets that might use CDN
os.environ['MPLBACKEND'] = 'Agg'  # Use non-interactive backend for matplotlib

# Ignore warnings related to widgets and CDN
warnings.filterwarnings('ignore', message=".*widget.*|.*CDN.*|.*SSL.*")

# 2. Configure memory limit to avoid OOM
try:
    import resource
    # Soft limit of 12GB (adjust according to available memory)
    soft, hard = resource.getrlimit(resource.RLIMIT_AS)
    mem_limit = 12 * (1024**3)  # 12GB in bytes
    resource.setrlimit(resource.RLIMIT_AS, (mem_limit, hard))
    print(f"✅ Memory limit set: 12GB")
except Exception:
    print("⚠️ Could not set memory limit")

# 3. Function to free memory (use it when you notice slowdowns)
def clean_memory():
    """Releases memory to prevent kernel crashes"""
    import gc
    print("🧹 Cleaning memory...")
    
    # Garbage collection
    gc.collect()
    
    # Release GPU cache if PyTorch is available
    try:
        import torch
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
            print("  ✓ GPU cache released")
    except ImportError:
        pass
    
    # Close matplotlib figures
    try:
        import matplotlib.pyplot as plt
        plt.close('all')
        print("  ✓ Figures closed")
    except ImportError:
        pass
        
    print("✅ Memory released")

print("✅ Anti-blocking configuration successfully applied")
print("💡 Use clean_memory() if you notice the notebook slowing down")
# ▶️ Memory monitor and safe execution
import gc
import time
import pickle
from pathlib import Path

# Create directory for checkpoints
CHECKPOINT_DIR = Path('./checkpoints')
CHECKPOINT_DIR.mkdir(exist_ok=True, parents=True)

# Define MVP mode - set to True for minimal viable product run (faster execution)
MVP_MODE = True

print("""
🔄 CHECKPOINT SYSTEM ACTIVE

The notebook uses a robust checkpoint system that allows recovery from crashes:
- Each of the {'5' if not MVP_MODE else '1'} experiments ({len(EXPERIMENTS) if 'EXPERIMENTS' in globals() else '5'} architectures × {'5' if not MVP_MODE else '1'} folds) is saved individually
- Training automatically resumes from the last saved checkpoint
- Perfect for long-running experiments that might be interrupted
""")

class SafeExecution:
    """
    Robust execution system to protect against crashes during training
    
    Features:
    - Automatic saving of trained models and metrics
    - Recovery from previous checkpoints if training was interrupted
    - Memory cleanup before each experiment
    
    Note: Ideal for long-running notebooks with multiple experiments
    """
    
    @staticmethod
    def save_checkpoint(data, name):
        """Save data in a checkpoint"""
        try:
            path = CHECKPOINT_DIR / f"{name}.pkl"
            with open(path, 'wb') as f:
                pickle.dump(data, f)
            print(f"✅ Checkpoint saved: {path}")
            return True
        except Exception as e:
            print(f"❌ Error saving checkpoint: {e}")
            return False
    
    @staticmethod
    def load_checkpoint(name):
        """Load data from a checkpoint"""
        try:
            path = CHECKPOINT_DIR / f"{name}.pkl"
            if not path.exists():
                return None
            
            with open(path, 'rb') as f:
                data = pickle.load(f)
            print(f"✅ Checkpoint loaded: {path}")
            return data
        except Exception as e:
            print(f"❌ Error loading checkpoint: {e}")
            return None
    
    @staticmethod
    def run_experiment(exp_name, fold=None):
        """
        Run an experiment safely with automatic checkpoint recovery
        
        Args:
            exp_name: Name of the experiment to run (must be in EXPERIMENTS)
            fold: Specific fold to run, or None to run all folds
            
        Note:
            When in MVP_MODE, this will only run fold F1 regardless of what's specified
        """
        if exp_name not in EXPERIMENTS:
            print(f"❌ Experiment '{exp_name}' does not exist")
            return
        
        # Determine folds to run
        if fold:
            folds_to_run = [fold] if fold in FOLDS else []
        else:
            folds_to_run = list(FOLDS.keys())
        
        if not folds_to_run:
            if MVP_MODE and fold not in FOLDS:
                print(f"❌ Fold '{fold}' not available in MVP_MODE (only {list(FOLDS.keys())} available)")
            else:
                print(f"❌ Invalid fold '{fold}'")
            return
            
        print(f"🔄 Running experiment {exp_name} on folds: {', '.join(folds_to_run)}")
        
        for current_fold in folds_to_run:
            # Checkpoint name for this experiment/fold
            checkpoint_name = f"{exp_name}_{current_fold}_result"
            
            # Check if a previous result exists
            checkpoint_data = SafeExecution.load_checkpoint(checkpoint_name)
            if checkpoint_data:
                model, history, best_rmse = checkpoint_data
                print(f"✅ Using previous result: RMSE = {best_rmse:.4f}")
                
                # Register global result
                if 'RESULTS' in globals():
                    RESULTS.append({
                        'exp': exp_name,
                        'fold': current_fold,
                        'rmse': best_rmse
                    })
                    
                # Update global histories
                if 'ALL_HISTORIES' in globals():
                    if exp_name not in ALL_HISTORIES:
                        ALL_HISTORIES[exp_name] = {}
                    ALL_HISTORIES[exp_name][current_fold] = history
                
                continue
            
            # If no checkpoint, run the training
            try:
                # Free memory before starting
                clean_memory()
                
                # Get configuration and build dataloaders
                print(f"🔄 Preparing data for fold {current_fold}")
                cfg = EXPERIMENTS[exp_name]
                val_year = FOLDS[current_fold]
                
                # Use reduced batch size for greater stability
                batch_size = max(8, BATCH_SIZE // 2)  # Half the original batch size, minimum 8
                train_loader, val_loader, in_dim = build_dataloaders(val_year, cfg['use_lags'], batch_size)
                
                # Adjust dropout according to documentation
                dropout = 0.25 if current_fold in ['F4', 'F5'] else 0.20
                
                # Create model
                model = MODEL_FACTORY[cfg['model']](in_dim, dropout=dropout).to(DEVICE)
                
                # Train model with error handling
                print(f"🔄 Training {exp_name} on fold {current_fold}")
                try:
                    model, history, best_rmse = train_with_history(
                        model, train_loader, val_loader,
                        epochs=60, patience=20,
                        lr=1e-3, weight_decay=1e-4,
                        fold=current_fold, exp_name=exp_name
                    )
                    
                    # Save checkpoint
                    SafeExecution.save_checkpoint(
                        (model, history, best_rmse),
                        checkpoint_name
                    )
                    
                    # Register global result
                    if 'RESULTS' in globals():
                        RESULTS.append({
                            'exp': exp_name,
                            'fold': current_fold,
                            'rmse': best_rmse
                        })
                    
                    # Update global histories
                    if 'ALL_HISTORIES' in globals():
                        if exp_name not in ALL_HISTORIES:
                            ALL_HISTORIES[exp_name] = {}
                        ALL_HISTORIES[exp_name][current_fold] = history
                        
                    print(f"✅ Training completed: RMSE = {best_rmse:.4f}")
                    
                except Exception as e:
                    print(f"❌ Error in training: {e}")
                
            except Exception as e:
                print(f"❌ Error in experiment {exp_name}, fold {current_fold}: {e}")
                continue
        
        print(f"✅ Experiment {exp_name} completed")

# Function to display saved results
def show_results():
    """Displays a table of results with experiments executed so far"""
    import pandas as pd
    
    # Search for results in checkpoints
    results = []
    
    for file in CHECKPOINT_DIR.glob("*_result.pkl"):
        try:
            parts = file.stem.split('_')
            exp = parts[0] 
            fold = parts[1]
            
            checkpoint = SafeExecution.load_checkpoint(f"{exp}_{fold}_result")
            if checkpoint:
                _, _, rmse = checkpoint
                results.append({
                    'exp': exp,
                    'fold': fold,
                    'rmse': rmse
                })
        except Exception:
            continue
    
    if results:
        df = pd.DataFrame(results)
        table = df.pivot(index='exp', columns='fold', values='rmse')
        display(table)
        
        # Show progress
        total = len(EXPERIMENTS) * len(FOLDS)
        completed = len(results)
        
        print(f"\n📊 Progress: {completed}/{total} ({completed/total:.1%})")
        
        if MVP_MODE:
            print(f"\n🚀 MVP Mode: Only showing results for fold F1 (most recent data)")
    else:
        print("❌ No saved results found")


print("""✅ Safe execution system activated

To run experiments safely:

  1. SafeExecution.run_experiment('GRU-ED', fold='F1')  # A specific fold
  2. SafeExecution.run_experiment('GRU-ED')             # All folds (in MVP mode: only F1)
  3. show_results()                                     # View saved results

Results are automatically saved and can be recovered
if the kernel dies during execution.

Current mode: {"🚀 MVP (F1 only)" if MVP_MODE else "📊 FULL (all folds)"}
""")


# ▶️ Environment setup (PyTorch + TF + XGBoost)
import sys, os, logging, warnings, json
from pathlib import Path
import platform, multiprocessing

try:
    import psutil
except ImportError:
    print("Warning: psutil is not installed. Hardware detection will be limited.")
    # Create a simple fallback class for psutil
    class PsutilFallback:
        @staticmethod
        def cpu_count(logical=True):
            return multiprocessing.cpu_count()
        
        @staticmethod
        def virtual_memory():
            class MemInfo:
                total = 8 * (1024**3)  # Assume 8GB
                available = 4 * (1024**3)  # Assume 4GB available
                percent = 50.0
            return MemInfo()
        
        @staticmethod
        def cpu_percent(*args, **kwargs):
            return 50.0
    
    psutil = PsutilFallback()

# Import main libraries with exception handling
try:
    import torch
    import numpy as np
    import pandas as pd
except ImportError as e:
    print(f"Critical error: {e}")
    print("Please run the robust dependency installation cell first")

# Additional imports are done with try/except to avoid kernel failures
try:
    import xarray as xr
    import pytorch_lightning as pl
    import tensorflow as tf
    import geopandas as gpd
    import cartopy.crs as ccrs
    import time
    from sklearn.preprocessing import RobustScaler, StandardScaler
    from torch.utils.data import Dataset, DataLoader
    from torch.optim.lr_scheduler import OneCycleLR
    from torch import nn
    from tqdm.auto import tqdm
    from IPython.display import display, HTML
    # ▶️ Functions for learning curves and prediction visualization
    import matplotlib.pyplot as plt
    import matplotlib.colors as mcolors
    from matplotlib.gridspec import GridSpec
    import cartopy.feature as cfeature
    from sklearn.metrics import mean_absolute_percentage_error
except ImportError as e:
    print(f"Warning: Could not import all dependencies: {e}")
    print("Some features may not be available")

print("Main imports loaded correctly")

# ▶️ Advanced environment detection and automatic resource configuration
def detect_environment():
    """
    Detects and configures optimal resources for training based on available hardware.
    Returns a dictionary of configurations to optimize performance.
    """
    env_info = {}
    
    # System information
    env_info['system'] = {
        'os': platform.system(),
        'platform': platform.platform(),
        'python': platform.python_version()
    }
    
    # CPU information
    cpu_count = multiprocessing.cpu_count()
    env_info['cpu'] = {
        'cores_logical': cpu_count,
        'cores_physical': psutil.cpu_count(logical=False) or cpu_count,
        'frequency': psutil.cpu_freq().max if psutil.cpu_freq() else 'Unknown',
        'usage_percent': psutil.cpu_percent(interval=0.1)
    }
    
    # RAM memory information
    memory = psutil.virtual_memory()
    env_info['memory'] = {
        'total_gb': round(memory.total / (1024**3), 2),
        'available_gb': round(memory.available / (1024**3), 2),
        'used_percent': memory.percent
    }
    
    # PyTorch and GPU configuration
    env_info['torch'] = {
        'version': torch.__version__,
        'cuda_available': torch.cuda.is_available(),
        'cudnn_enabled': torch.backends.cudnn.enabled,
        'gpu_count': torch.cuda.device_count()
    }
    
    # GPU details if available
    if torch.cuda.is_available():
        gpus = []
        for i in range(torch.cuda.device_count()):
            gpu_props = torch.cuda.get_device_properties(i)
            gpus.append({
                'name': gpu_props.name,
                'memory_gb': round(gpu_props.total_memory / (1024**3), 2),
                'compute_capability': f"{gpu_props.major}.{gpu_props.minor}",
                'multi_processor_count': gpu_props.multi_processor_count
            })
        env_info['gpu'] = gpus
    
    # Automatically determine optimal configuration
    config = auto_configure_resources(env_info)
    env_info['optimized_config'] = config
    
    return env_info

def auto_configure_resources(env_info):
    """
    Automatically configures parameters to optimize performance
    based on available hardware.
    """
    config = {}
    
    # Compute device (GPU or CPU)
    config['device'] = 'cuda' if env_info['torch']['cuda_available'] else 'cpu'
    
    # Worker configuration
    if config['device'] == 'cuda':
        # For GPU: Fewer workers to avoid bottlenecks in data transfer
        recommended_workers = min(4, env_info['cpu']['cores_logical'] // 2)
    else:
        # For CPU: More workers to parallelize data loading
        recommended_workers = max(4, env_info['cpu']['cores_logical'] - 2)
    
    # Adjust workers based on available memory
    mem_factor = env_info['memory']['available_gb'] / 16.0  # Normalize to 16GB
    recommended_workers = min(recommended_workers, int(recommended_workers * mem_factor) + 1)
    config['num_workers'] = max(1, recommended_workers)  # At least 1 worker
    
    # Configure prefetch_factor based on memory
    config['prefetch_factor'] = 2 if env_info['memory']['available_gb'] < 8 else 4
    
    # Automatic batch size
    if config['device'] == 'cuda':
        # If GPU available, based on GPU memory
        total_gpu_mem = sum(gpu['memory_gb'] for gpu in env_info['gpu'])
        if total_gpu_mem > 10:
            config['batch_size'] = 128
        elif total_gpu_mem > 6:
            config['batch_size'] = 64
        else:
            config['batch_size'] = 32
    else:
        # If CPU only, smaller batch size
        if env_info['memory']['available_gb'] > 12:
            config['batch_size'] = 32
        else:
            config['batch_size'] = 16
    
    # Automatic optimizations
    if config['device'] == 'cuda':
        # Enable CUDA optimizations
        torch.backends.cudnn.benchmark = True
        
        # Enable TF32 on Ampere+ GPUs (compute capability >= 8.0)
        has_ampere = any(float(gpu['compute_capability']) >= 8.0 for gpu in env_info['gpu'])
        if has_ampere:
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            config['use_tf32'] = True
        
        # Configure mixed precision usage automatically
        config['use_amp'] = True
        
        # Memory optimizations
        config['pin_memory'] = True
        config['non_blocking'] = True
    
    return config

# Detect environment and apply optimized configurations
ENV_INFO = detect_environment()
CONFIG = ENV_INFO['optimized_config']

# Update global variables with optimized configuration
DEVICE = torch.device(CONFIG['device'])
N_GPU = torch.cuda.device_count()
CPU_CORES = ENV_INFO['cpu']['cores_logical']
NUM_WORKERS = CONFIG['num_workers']
BATCH_SIZE = CONFIG['batch_size']

# Activate GPU optimizations if available
if DEVICE.type == 'cuda':
    # Enable asynchronous operations - fixed to use device index
    if DEVICE.index is not None:
        torch.cuda.set_device(DEVICE.index)
    else:
        # If no index specified, use device 0
        torch.cuda.set_device(0)
    
    # Automatic mixed precision 
    if CONFIG.get('use_amp', False):
        # Will be activated in training functions
        pass

    # Enable optimizations for tensor types
    torch.set_float32_matmul_precision('high')

# Show system information and optimized configuration
print("\n" + "="*50)
print("📊 ENVIRONMENT DETECTION AND OPTIMIZATION")
print("="*50)

print("\n📋 System Summary:")
print(f"🖥️  OS: {ENV_INFO['system']['platform']}")
print(f"🧠 CPU: {ENV_INFO['cpu']['cores_physical']} physical cores / {ENV_INFO['cpu']['cores_logical']} logical cores")
print(f"💾 Memory: {ENV_INFO['memory']['available_gb']:.1f}GB available of {ENV_INFO['memory']['total_gb']:.1f}GB total")

if ENV_INFO['torch']['cuda_available']:
    print("\n🔥 Detected GPUs:")
    for i, gpu in enumerate(ENV_INFO['gpu']):
        print(f"   GPU {i}: {gpu['name']} ({gpu['memory_gb']:.1f}GB, {gpu['compute_capability']})")
        
    # Show initial memory information
    mem_allocated = torch.cuda.memory_allocated() / (1024**3)
    mem_reserved = torch.cuda.memory_reserved() / (1024**3)
    print(f"   Initial GPU memory: {mem_allocated:.2f}GB used / {mem_reserved:.2f}GB reserved")
else:
    print("\n❌ No GPUs detected - Using CPU")

print("\n⚙️ Optimized Training Configuration:")
print(f"📱 Device: {CONFIG['device'].upper()}")
print(f"👥 Workers: {CONFIG['num_workers']} (of {ENV_INFO['cpu']['cores_logical']} available)")
print(f"📦 Batch Size: {CONFIG['batch_size']}")
if CONFIG['device'] == 'cuda':
    print(f"🚀 Mixed Precision: {'Enabled' if CONFIG.get('use_amp', False) else 'Disabled'}")
    print(f"⚡ TF32: {'Available' if CONFIG.get('use_tf32', False) else 'Not available'}")
    print(f"🔄 Pin Memory: {'Enabled' if CONFIG.get('pin_memory', False) else 'Disabled'}")
    print(f"⏩ Non-blocking Transfers: {'Enabled' if CONFIG.get('non_blocking', False) else 'Disabled'}")

print(f"\n✅ Environment automatically configured for optimal performance")
print("="*50)

# ▶️ Path configuration (Colab vs Local)
from pathlib import Path
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    BASE_PATH = Path('/content/drive/MyDrive/ml_precipitation_prediction')
else:
    BASE_PATH = Path.cwd()
    # climb to project root if inside subfolder
    for p in [BASE_PATH, *BASE_PATH.parents]:
        if (p / '.git').exists():
            BASE_PATH = p; break
    DEBUG_MODE = True
    SAFE_LOCAL_MODE = True
    BATCH_SIZE = 8
    NUM_WORKERS = 0
    INPUT_WINDOW = 24  # Reducido
    HORIZON = 6        # Reducido
print('BASE_PATH =', BASE_PATH)

# centralised dataset / model paths
DATA_DIR      = BASE_PATH/'data'/'output'
MODEL_DIR     = BASE_PATH/'models'/'output'/'trained_models'; MODEL_DIR.mkdir(parents=True, exist_ok=True)
IMAGE_DIR     = MODEL_DIR/'images'; IMAGE_DIR.mkdir(exist_ok=True)
FEATURES_NC   = BASE_PATH/'models'/'output'/'features_fusion_branches.nc'
FULL_NC       = DATA_DIR/'complete_dataset_with_features_with_clusters_elevation_with_windows.nc'
print('Using FULL_NC  :', FULL_NC)
print('Using FEATURES :', FEATURES_NC)

# Actualizar diccionario de experimentos según la nueva nomenclatura
EXPERIMENTS = {
    'GRU-ED': {'model':'gru_ed', 'use_lags':False},
    'GRU-ED-PAFC': {'model':'gru_ed', 'use_lags':True},
    'AE-FUSION-GRU-ED-PAFC': {'model':'ae_fusion_gru', 'use_lags':True},
    'AE-FUSION-GRU-ED-PAFC-T': {'model':'ae_fusion_gru_t', 'use_lags':True},
    'AE-FUSION-GRU-ED-PAFC-T-TopoMask': {'model':'ae_fusion_gru_t_mask', 'use_lags':True},
}

# ▶️ Add variable definitions consistent with documentation
FULL_FEATURES = [
    'precip_hist','lag_1','lag_2','lag_12',
    'month_sin','month_cos','doy_sin','doy_cos',
    'elevation','slope','roughness','curvature','aspect',
    'alt_cluster','ceemdan_imf1','ceemdan_imf2','ceemdan_imf3',
    'tvfemd_imf1','tvfemd_imf2','tvfemd_imf3'
]

BASE_FEATURES = [
    'total_precipitation',  # en lugar de 'precip_hist'
    'total_precipitation_lag1', 'total_precipitation_lag2', 'total_precipitation_lag12',
    'month_sin', 'month_cos', 'doy_sin', 'doy_cos',
    'elevation', 'slope', 'aspect',
    'cluster_elevation'  # en lugar de 'alt_cluster'
]


# ▶️ Helper functions
import pandas as pd, numpy as np
def add_time_encodings(ds: xr.Dataset):
    '''Add month/day-of-year sinusoidal encodings'''
    dates = pd.to_datetime(ds['time'].values)
    month = dates.month
    doy = dates.dayofyear
    ds['month_sin'] = ('time', np.sin(2*np.pi*month/12))
    ds['month_cos'] = ('time', np.cos(2*np.pi*month/12))
    ds['doy_sin']   = ('time', np.sin(2*np.pi*doy/365.25))
    ds['doy_cos']   = ('time', np.cos(2*np.pi*doy/365.25))
    return ds

# ▶️ Logger & helper prints
logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s [%(levelname)s] %(message)s',
                    datefmt='%H:%M:%S')
logger = logging.getLogger('precip')

def print_progress(msg, level=0, is_start=False, is_end=False):
    prefix={0:'🔵 ' if is_start else '✅ ' if is_end else '➡️ ',
            1:'  ⚪ ',2:'    • '}.get(level,'')
    print(f'{prefix}{msg}')

# Enhanced version of print_progress with more features
def enhanced_logger(msg, level=0, is_start=False, is_end=False):
    """
    Enhanced logging function with timestamp and styling.
    
    Args:
        msg: Message to log
        level: Indentation level (0=main, 1=sub, 2=detail)
        is_start: Set to True for start messages (blue)
        is_end: Set to True for completion messages (green)
    """
    import time
    timestamp = time.strftime('%H:%M:%S')
    prefix = {0:'🔵 ' if is_start else '✅ ' if is_end else '➡️ ',
              1:'  ⚪ ', 2:'    • '}.get(level, '')
    print(f'[{timestamp}] {prefix}{msg}')

# (Reuse code from earlier minimal pipeline, but path variable FULL_NC)
DATASET_PATH = str(FULL_NC)
INPUT_WINDOW=48; HORIZON=12; BATCH_SIZE=32
FOLDS={'F1':2024,'F2':2023,'F3':2022,'F4':2000,'F5':1990}
# ... (insert PyTorch dataset, model, training utils from earlier) ...
print_progress('⚠️   PyTorch quick baseline section trimmed for brevity — insert from earlier if desired', level=1)

# ▶️ Verify precipitation lags utility
def verify_precipitation_lags(ds, required_lags=None, min_valid_ratio=0.9):
    all_possible = [f"total_precipitation_lag{i}" for i in [1,2,3,4,12,24,36]]
    lags = required_lags or [l for l in all_possible if l in ds.data_vars]
    if not lags: raise ValueError('No lag variables found.')
    for lag in lags:
        arr = ds[lag].values
        valid = np.count_nonzero(~np.isnan(arr))
        ratio = valid/arr.size
        logger.info(f'{lag}: {ratio:.1%} valid')
        if ratio<min_valid_ratio:
            raise ValueError(f'{lag} has only {ratio:.1%} valid data (<{min_valid_ratio})')
    logger.info('Lag verification ✅')

# ▶️ NaN‑robust scaling utils
def check_nans(arr, name='array'):
    nan_cnt=np.isnan(arr).sum(); tot=arr.size
    return {'name':name,'nan':nan_cnt,'total':tot,'pct':nan_cnt/tot*100,'has':nan_cnt>0}

def replace_nans(arr, strategy='mean'):
    if not np.isnan(arr).any(): return arr
    arr=arr.copy()
    if strategy=='mean':
        fill=np.nanmean(arr); arr[np.isnan(arr)]=fill
    elif strategy=='median':
        fill=np.nanmedian(arr); arr[np.isnan(arr)]=fill
    else:
        arr=np.nan_to_num(arr)
    return arr

class ScalerNaN:
    def fit(self,X):
        self.mean_=np.nanmean(X,0); var=np.nanvar(X,0); var[var<1e-9]=1
        self.scale_=np.sqrt(var); return self
    def transform(self,X):
        return (X-self.mean_)/self.scale_
    def fit_transform(self,X): self.fit(X); return self.transform(X)
    def inverse_transform(self,X):
        return X*self.scale_+self.mean_

# Direct import to avoid the "Dataset is not defined" error
from torch.utils.data import Dataset, DataLoader

# ▶️ Dataset & DataLoader builder
class PrecipDataset(Dataset):
    def __init__(self, ds, idx_list, input_window, horizon,
                 sc_p, sc_x, features, y_dim_name='latitude', x_dim_name='longitude'):
        self.ds = ds
        self.idx = idx_list
        self.w = input_window
        self.h = horizon
        self.scp = sc_p
        self.scx = sc_x
        self.features = features
        self.y_dim_name = y_dim_name if y_dim_name in ds.dims else 'y'
        self.x_dim_name = x_dim_name if x_dim_name in ds.dims else 'x'
        
        # Cache for static variables to avoid repeated access
        self._static_cache = {}
        
        # Pre-validate all available features
        self.valid_features = []
        for feat in features:
            if feat in ds.data_vars:
                self.valid_features.append(feat)
            else:
                print(f"Warning: Feature '{feat}' not found in the dataset")
                
        if len(self.valid_features) == 0:
            raise ValueError("No valid features found in the dataset")
            
        print(f"Valid features: {self.valid_features}")

    def __len__(self):
        return len(self.idx)

    def __getitem__(self, i):
        # Unpack indices with correct dimension names
        t, y_idx, x_idx = self.idx[i]
        
        try:
            # Get window data
            window_data = self.ds.isel(time=slice(t - self.w, t), 
                                    **{self.y_dim_name: y_idx, self.x_dim_name: x_idx})

            # Get target data
            target_data = self.ds.isel(time=slice(t, t + self.h), 
                                     **{self.y_dim_name: y_idx, self.x_dim_name: x_idx})
            
            # Extract and prepare the 'total_precipitation' target
            if 'total_precipitation' in target_data:
                tgt_values = target_data['total_precipitation'].values
                if not np.issubdtype(tgt_values.dtype, np.number):
                    tgt_values = tgt_values.astype(np.float32)
                # Replace NaNs with zeros
                tgt = np.nan_to_num(tgt_values, nan=0.0).astype(np.float32)
            else:
                # If no target precipitation, use zeros
                tgt = np.zeros(self.h, dtype=np.float32)

            # List to store features
            input_features_list = []
            
            for var in self.valid_features:
                try:
                    if var == 'total_precipitation':
                        # Historical precipitation
                        if var in window_data:
                            values = window_data[var].values
                            if not np.issubdtype(values.dtype, np.number):
                                values = values.astype(np.float32)
                            
                            # Replace NaNs with zeros
                            values = np.nan_to_num(values, nan=0.0)
                            feature_ts = self.scp.transform(values.reshape(-1, 1))
                            input_features_list.append(feature_ts)
                        else:
                            # If variable doesn't exist, add zeros
                            zeros = np.zeros((self.w, 1), dtype=np.float32)
                            input_features_list.append(zeros)
                            
                    elif var.startswith('total_precipitation_lag'):
                        # Lag variables
                        if var in window_data:
                            values = window_data[var].values
                            if not np.issubdtype(values.dtype, np.number):
                                values = values.astype(np.float32)
                            
                            # Replace NaNs with zeros
                            values = np.nan_to_num(values, nan=0.0)
                            feature_ts = self.scp.transform(values.reshape(-1, 1))
                            input_features_list.append(feature_ts)
                        else:
                            # If variable doesn't exist, add zeros
                            zeros = np.zeros((self.w, 1), dtype=np.float32)
                            input_features_list.append(zeros)
                            
                    elif var in window_data:
                        # Other variables (temporal or static)
                        cache_key = f"{var}_{y_idx}_{x_idx}"
                        
                        if 'time' in window_data[var].dims:
                            # Temporal variable
                            values = window_data[var].values
                            if not np.issubdtype(values.dtype, np.number):
                                values = values.astype(np.float32)
                            
                            # Replace NaNs with zeros
                            values = np.nan_to_num(values, nan=0.0)
                            feature_ts = self.scx.transform(values.reshape(-1, 1))
                            input_features_list.append(feature_ts)
                        else:
                            # Static variable - use cache if available
                            if cache_key in self._static_cache:
                                feature_ts = self._static_cache[cache_key]
                            else:
                                static_val = window_data[var].values
                                
                                # Ensure it's a numeric array
                                if not isinstance(static_val, (np.ndarray, np.number)):
                                    static_val = np.array([0.0], dtype=np.float32)
                                elif not np.issubdtype(static_val.dtype, np.number):
                                    static_val = np.array([0.0], dtype=np.float32)
                                else:
                                    static_val = np.asarray(static_val, dtype=np.float32)
                                
                                # Handle NaNs
                                static_val = np.nan_to_num(static_val, nan=0.0)
                                
                                # Ensure correct shape for transformation
                                static_val = static_val.reshape(-1, 1)
                                
                                # Transform and repeat for all timesteps
                                try:
                                    transformed = self.scx.transform(static_val)
                                    feature_ts = np.repeat(transformed, self.w).reshape(self.w, -1)
                                except Exception:
                                    # In case of error, use zero values
                                    feature_ts = np.zeros((self.w, 1), dtype=np.float32)
                                
                                # Save in cache
                                self._static_cache[cache_key] = feature_ts
                                
                            input_features_list.append(feature_ts)
                except Exception as e:
                    # If there's an error with a specific feature, use zeros as fallback
                    print(f"Error with feature {var}: {str(e)}")
                    feature_ts = np.zeros((self.w, 1), dtype=np.float32)
                    input_features_list.append(feature_ts)
            
            # If no features, use zero vector
            if not input_features_list:
                X_fallback = np.zeros((self.w, len(self.features)), dtype=np.float32)
                return torch.tensor(X_fallback, dtype=torch.float32), torch.tensor(tgt, dtype=torch.float32)
            
            # Concatenate features
            X = np.hstack(input_features_list).astype(np.float32)
            
            return torch.tensor(X, dtype=torch.float32), torch.tensor(tgt, dtype=torch.float32)
        
        except Exception as e:
            # Global error handling - report but return fallback tensors
            print(f"Error processing sample {i}, indices: {t},{y_idx},{x_idx}: {str(e)}")
            X_fallback = np.zeros((self.w, len(self.features)), dtype=np.float32)
            y_fallback = np.zeros(self.h, dtype=np.float32)
            return torch.tensor(X_fallback, dtype=torch.float32), torch.tensor(y_fallback, dtype=torch.float32)

def build_dataloaders(val_year, use_lags, batch_size=BATCH_SIZE):
    # Open and verify the dataset before using it
    try:
        ds = xr.open_dataset(DATASET_PATH)
        
        # Verify basic dimensions
        required_dims = ['time']
        for dim in required_dims:
            if dim not in ds.dims:
                raise ValueError(f"Dataset does not contain the required dimension: {dim}")
        
        # Ensure there is valid data
        if ds.dims['time'] < INPUT_WINDOW + HORIZON:
            raise ValueError(f"Dataset does not contain enough time steps. At least {INPUT_WINDOW + HORIZON} required")
            
        # Add time encodings
        ds = add_time_encodings(ds)
    
    except Exception as e:
        raise RuntimeError(f"Error loading or processing the dataset: {str(e)}")
    
    # Diagnostic information
    print(f"Dataset loaded: variables {list(ds.data_vars.keys())[:10]}...")
    print(f"Dimensions: {ds.dims}")

    train_start = np.datetime64(f'{val_year-4}-01-01')
    train_end   = np.datetime64(f'{val_year-1}-12-31')
    val_start   = np.datetime64(f'{val_year}-01-01')
    val_end     = np.datetime64(f'{val_year}-12-31')

    train_mask = (ds['time']>=train_start)&(ds['time']<=train_end)
    val_mask   = (ds['time']>=val_start)&(ds['time']<=val_end)

    # Ensure total_precipitation exists
    if 'total_precipitation' not in ds.data_vars:
        raise ValueError(f"'total_precipitation' is not in the dataset. Available variables: {list(ds.data_vars.keys())}")
    
    # Extract precipitation values and preprocess with robust handling
    precip_values = ds['total_precipitation'].where(train_mask).values
    
    # Robust data verification and cleaning
    try:
        # Convert to float32 and handle NaNs
        precip_values = precip_values.astype(np.float32)
        precip_values = np.nan_to_num(precip_values, nan=0.0)
        
        # Verify that there are valid values
        if np.all(precip_values == 0) or np.all(np.isnan(precip_values)):
            print("WARNING: All precipitation values are zero or NaN")
            # Add a small noise to avoid divisions by zero
            precip_values = precip_values + np.random.normal(0, 0.001, precip_values.shape)
    except Exception as e:
        print(f"Error processing precipitation values: {str(e)}")
        print("Using fallback values...")
        # Create fallback values
        precip_values = np.random.normal(0, 1.0, (100, 100)).astype(np.float32)
    
    # Fit RobustScaler with error handling
    try:
        sc_p = RobustScaler().fit(precip_values.reshape(-1, 1))
    except Exception:
        print("Error fitting RobustScaler for precipitation. Using StandardScaler as fallback.")
        sc_p = StandardScaler().fit(precip_values.reshape(-1, 1))
    
    # Numeric variables for StandardScaler with robust handling
    numeric_vars = []
    preds = []
    
    # List of variables we know are numeric
    potential_numeric_vars = ['month_sin', 'month_cos', 'doy_sin', 'doy_cos', 
                             'elevation', 'slope', 'aspect']
    
    # Verify availability and type of each variable
    for var in potential_numeric_vars:
        if var in ds.data_vars:
            try:
                # Extract values and verify type
                var_values = ds[var].where(train_mask).values
                
                # Clean and verify
                var_values = np.nan_to_num(var_values, nan=0.0).astype(np.float32)
                
                if np.issubdtype(var_values.dtype, np.number):
                    numeric_vars.append(var)
                    preds.append(var_values.flatten())
            except Exception as e:
                print(f"Error processing variable {var}: {str(e)}")
    
    print(f"Numeric variables used for StandardScaler: {numeric_vars}")
    
    if not preds:
        print("WARNING: No numeric variables found for StandardScaler")
        # Create fallback data for StandardScaler
        dummy_data = np.random.normal(0, 1.0, 1000).reshape(-1, 1)
        sc_x = StandardScaler().fit(dummy_data)
    else:
        try:
            all_data = np.concatenate(preds).reshape(-1, 1)
            # Clean data
            all_data = np.nan_to_num(all_data, nan=0.0)
            sc_x = StandardScaler().fit(all_data)
        except Exception:
            print("Error fitting StandardScaler. Using simple fallback.")
            sc_x = StandardScaler().fit(np.random.normal(0, 1.0, 1000).reshape(-1, 1))
    
    def make_idx(mask):
        idx = []
        y_dim_name = 'latitude' if 'latitude' in ds.sizes else 'y'
        x_dim_name = 'longitude' if 'longitude' in ds.sizes else 'x'
        
        if y_dim_name not in ds.sizes or x_dim_name not in ds.sizes:
            raise ValueError(f"Dataset must contain dimensions '{y_dim_name}' and '{x_dim_name}'. Found: {list(ds.sizes.keys())}")
        
        for t in range(INPUT_WINDOW, len(ds['time']) - HORIZON):
            if t + HORIZON - 1 < len(mask) and mask[t + HORIZON - 1]:
                for y in range(ds.sizes[y_dim_name]):
                    for x in range(ds.sizes[x_dim_name]):
                        idx.append((t, y, x))
        return idx
    
    train_idx = make_idx(train_mask)
    val_idx = make_idx(val_mask)
    
    print(f"Training examples: {len(train_idx)}")
    print(f"Validation examples: {len(val_idx)}")
    
    # Build feature list with robust verification
    feats = []
    
    # Always include 'total_precipitation'
    feats.append('total_precipitation')
    
    # Include lags if requested
    if use_lags:
        for lag in ['total_precipitation_lag1', 'total_precipitation_lag2', 'total_precipitation_lag12']:
            if lag in ds.data_vars:
                feats.append(lag)
            else:
                print(f"Warning: Lag '{lag}' not available in the dataset")
    
    # Include temporal encoding variables
    for var in ['month_sin', 'month_cos', 'doy_sin', 'doy_cos']:
        if var in ds.data_vars:
            feats.append(var)
    
    # Include topographic variables with safe verification
    for var in ['elevation', 'slope', 'aspect', 'cluster_elevation']:
        if var in ds.data_vars:
            try:
                # Verify it's accessible
                test_val = ds[var].isel(
                    **{dim: 0 for dim in ds[var].dims if dim != 'time'}
                ).values
                
                # Handle data types
                test_val = np.nan_to_num(test_val, nan=0.0).astype(np.float32)
                
                # If we get here, the variable is usable
                feats.append(var)
            except Exception as e:
                print(f"Error verifying variable '{var}': {str(e)}")
    
    print(f"Final features for the model: {feats}")
    
    # Configure logging level to reduce warnings
    warnings.filterwarnings('ignore', message="Feature.*not found in dataset slice")
    warnings.filterwarnings('ignore', message="invalid value encountered in divide")
    warnings.filterwarnings('ignore', message="overflow encountered in reduce")
    
    # Create datasets
    train_ds = PrecipDataset(ds, train_idx, INPUT_WINDOW, HORIZON, sc_p, sc_x, feats)
    val_ds = PrecipDataset(ds, val_idx, INPUT_WINDOW, HORIZON, sc_p, sc_x, feats)
    
    # Verify real dimension by testing a batch with error handling
    try:
        sample_loader = DataLoader(train_ds, batch_size=1)
        X_sample, _ = next(iter(sample_loader))
        real_feature_dim = X_sample.shape[2]
        print(f"Real input dimension: {real_feature_dim}")
    except Exception:
        print("Error determining real dimension. Using number of features as estimate.")
        real_feature_dim = len(feats)
    
    # Adjust workers based on system and stability
    safe_workers = min(2, NUM_WORKERS)  # Limit to maximum 2 workers to avoid issues
    print(f"Using {safe_workers} workers for DataLoaders")
    
    # Final DataLoaders
    train_loader = DataLoader(
        train_ds, 
        batch_size=batch_size, 
        shuffle=True,
        num_workers=safe_workers, 
        pin_memory=True,
        persistent_workers=safe_workers > 0,
        prefetch_factor=2 if safe_workers > 0 else None
    )
    
    val_loader = DataLoader(
        val_ds, 
        batch_size=batch_size, 
        shuffle=False,
        num_workers=safe_workers, 
        pin_memory=True,
        persistent_workers=safe_workers > 0,
        prefetch_factor=2 if safe_workers > 0 else None
    )
    
    return train_loader, val_loader, real_feature_dim


# ▶️ Model definitions
# Ensure nn is imported directly before defining the models
import torch.nn as nn

class GRUEncoderDecoder(nn.Module):
    def __init__(self, input_dim, hidden_size=128, num_layers=2, dropout=0.2, horizon=HORIZON):
        super().__init__()
        self.enc = nn.GRU(input_dim, hidden_size, num_layers,
                          batch_first=True, dropout=dropout)
        self.dec = nn.GRU(1, hidden_size, num_layers,
                          batch_first=True, dropout=dropout)
        self.fc  = nn.Linear(hidden_size,1)
        self.hor = horizon

    def forward(self, x, teacher_forcing_ratio=0.5, y=None):
        _, h = self.enc(x)
        dec_in = x[:, -1:, 0:1]
        outs=[]
        for t in range(self.hor):
            o, h = self.dec(dec_in, h)
            pred = self.fc(o.squeeze(1))
            outs.append(pred)
            if self.training and y is not None and torch.rand(1)<teacher_forcing_ratio:
                dec_in = y[:, t:t+1].unsqueeze(-1)
            else:
                dec_in = pred.unsqueeze(1)
        
        # Apilar las salidas y eliminar la última dimensión para pasar de [batch, horizon, 1] a [batch, horizon]
        return torch.stack(outs, dim=1).squeeze(-1)

# Implementation aligned with documentation
class Conv3DAutoEncoder(nn.Module):
    def __init__(self, in_channels=3, bottleneck_dim=64):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Conv3d(in_channels, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2),
            nn.Conv3d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool3d(kernel_size=2),
            nn.Conv3d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*6*6*6, bottleneck_dim)  # Adjust dimensions based on your input
        )
        
    def forward(self, x):
        return self.encoder(x)

class AEFusionGRU(nn.Module):
    def __init__(self, input_dim, hidden_size=128, num_layers=2, dropout=0.2, horizon=HORIZON):
        super().__init__()
        self.ae = Conv3DAutoEncoder(in_channels=3, bottleneck_dim=64)
        
        # Combined dim: original features + bottleneck
        combined_dim = input_dim + 64;
        
        self.backbone = GRUEncoderDecoder(combined_dim, hidden_size, num_layers, dropout, horizon)
    
    def forward(self, x, teacher_forcing_ratio=0.5, y=None):
        # Assuming x_imfs is processed elsewhere and passed with x
        # This is a placeholder for the actual implementation
        ae_features = torch.zeros((x.size(0), 64), device=x.device)
        
        # Concatenate features
        combined = torch.cat([x, ae_features.unsqueeze(1).expand(-1, x.size(1), -1)], dim=2)
        
        return self.backbone(combined, teacher_forcing_ratio, y)

class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hidden_dim, n_heads=4, dropout=0.1):
        super().__init__()
        self.attention = nn.MultiheadAttention(hidden_dim, n_heads, dropout=dropout)
        self.norm = nn.LayerNorm(hidden_dim)  # Corregido de LayerNormalization a LayerNorm
        
    def forward(self, x, mask=None):
        attn_out, _ = self.attention(x, x, x, attn_mask=mask)
        return self.norm(x + attn_out)

class AEFusionGRUT(AEFusionGRU):
    def __init__(self, input_dim, hidden_size=128, num_layers=2, dropout=0.2, horizon=HORIZON):
        super().__init__(input_dim, hidden_size, num_layers, dropout, horizon)
        self.attention = MultiHeadAttentionLayer(hidden_size, n_heads=4, dropout=dropout)
        
    def forward(self, x, teacher_forcing_ratio=0.5, y=None):
        # Similar implementation as AEFusionGRU but with attention
        # This is placeholder for the actual implementation with attention
        return super().forward(x, teacher_forcing_ratio, y)

class AEFusionGRUTMask(AEFusionGRUT):
    def __init__(self, input_dim, hidden_size=128, num_layers=2, dropout=0.2, horizon=HORIZON):
        super().__init__(input_dim, hidden_size, num_layers, dropout, horizon)
        
    def forward(self, x, teacher_forcing_ratio=0.5, y=None):
        # Similar implementation but with causal masking for attention
        # This is placeholder for the actual implementation with causal masking
        return super().forward(x, teacher_forcing_ratio, y)

# Update MODEL_FACTORY with proper implementations
MODEL_FACTORY = {
    'gru_ed': GRUEncoderDecoder,
    'ae_fusion_gru': AEFusionGRU,
    'ae_fusion_gru_t': AEFusionGRUT,
    'ae_fusion_gru_t_mask': AEFusionGRUTMask,
}


# ▶️ Training utilities
from torchmetrics.functional import mean_squared_error
def huber_weighted(preds, target):
    # Asegurar que preds y target tienen la misma forma
    if preds.dim() == 3 and preds.size(2) == 1:
        preds = preds.squeeze(-1)  # Convertir de [batch, seq, 1] a [batch, seq]
        
    # Crear pesos para cada horizonte (1 + h/12)
    h = torch.arange(1, target.size(1)+1, device=preds.device).float()
    weights = 1 + h/12.0
    
    # Calcular pérdida Huber
    loss = torch.nn.functional.huber_loss(preds, target, reduction='none')
    
    # Aplicar pesos al horizonte y promediar
    weighted_loss = loss * weights.view(1, -1)
    return weighted_loss.mean()

def train_one_epoch(model, loader, opt, tf_ratio, scheduler=None):
    model.train()
    losses=[]
    for X,y in loader:
        X,y = X.to(DEVICE), y.to(DEVICE)
        preds = model(X, teacher_forcing_ratio=tf_ratio, y=y)
        # Asegurar que preds tiene la forma correcta
        if preds.dim() == 3 and preds.size(2) == 1:
            preds = preds.squeeze(-1)
        loss = huber_weighted(preds, y)
        opt.zero_grad()
        loss.backward()
        opt.step()
        if scheduler:
            scheduler.step()
        losses.append(loss.item())
    return np.mean(losses)

@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    rmses=[]
    for X,y in loader:
        X,y = X.to(DEVICE), y.to(DEVICE)
        preds = model(X, teacher_forcing_ratio=0.0)
        # Asegurar que preds tiene la misma forma que y
        if preds.dim() == 3 and preds.size(2) == 1:
            preds = preds.squeeze(-1)  # Convertir de [batch, seq, 1] a [batch, seq]
        
        rmse = mean_squared_error(preds, y, squared=False)
        rmses.append(rmse.item())
    return np.mean(rmses)

# Update the training function to handle shapes correctly
def train_with_history(model, train_loader, val_loader, epochs=60, patience=20, 
                      lr=1e-3, weight_decay=1e-4, fold='', exp_name=''):
    print_progress(f"Starting training of {exp_name} on fold {fold}", is_start=True)
    
    # Force model to be on GPU
    model = model.to(DEVICE)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = OneCycleLR(optimizer, max_lr=lr, total_steps=epochs*len(train_loader),
                         pct_start=0.3, anneal_strategy='cos')
    
    # Create GradScaler with updated syntax
    if torch.cuda.is_available():
        scaler = torch.amp.GradScaler('cuda')
    else:
        scaler = None
    use_amp = scaler is not None
    
    # Initialize history for learning curves
    history = {
        'train_loss': [],
        'val_loss': [],
        'val_rmse': [],
        'learning_rate': [],
        'teacher_forcing': []
    }
    
    best_rmse = float('inf')
    best_model_state = None
    counter = 0
    
    for epoch in range(1, epochs+1):
        # Run between celdas to monitor
        aggressive_memory_cleanup()
        gpu_monitor()
        # Calculate teacher forcing ratio with cosine decay (0.7→0.3)
        tf_ratio = 0.7 - (epoch-1)*(0.4)/(epochs-1)
        history['teacher_forcing'].append(tf_ratio)
        
        # Training
        model.train()
        train_losses = []
        for X, y in train_loader:
            X, y = X.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
            optimizer.zero_grad(set_to_none=True)  # More efficient than zero_grad()
            
            # Use mixed precision for forward pass if available
            if use_amp:
                with torch.autocast(device_type='cuda'):
                    preds = model(X, teacher_forcing_ratio=tf_ratio, y=y)
                    if preds.dim() == 3 and preds.size(2) == 1:
                        preds = preds.squeeze(-1)
                    loss = huber_weighted(preds, y)
                
                # Scale gradients and optimize with mixed precision
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                # Normal flow if no AMP
                preds = model(X, teacher_forcing_ratio=tf_ratio, y=y)
                if preds.dim() == 3 and preds.size(2) == 1:
                    preds = preds.squeeze(-1)
                loss = huber_weighted(preds, y)
                loss.backward()
                optimizer.step()
            
            scheduler.step()
            train_losses.append(loss.item())
        
        # Get current learning rate
        current_lr = scheduler.get_last_lr()[0]
        history['learning_rate'].append(current_lr)
        
        # Evaluation
        model.eval()
        val_losses = []
        val_rmses = []
        with torch.no_grad():
            for X, y in val_loader:
                X, y = X.to(DEVICE, non_blocking=True), y.to(DEVICE, non_blocking=True)
                
                if use_amp:
                    with torch.autocast(device_type='cuda'):
                        preds = model(X, teacher_forcing_ratio=0)
                        if preds.dim() == 3 and preds.size(2) == 1:
                            preds = preds.squeeze(-1)
                        val_loss = huber_weighted(preds, y).item()
                        val_rmse = mean_squared_error(preds, y, squared=False).item()
                else:
                    preds = model(X, teacher_forcing_ratio=0)
                    if preds.dim() == 3 and preds.size(2) == 1:
                        preds = preds.squeeze(-1)
                    val_loss = huber_weighted(preds, y).item()
                    val_rmse = mean_squared_error(preds, y, squared=False).item()
                
                val_losses.append(val_loss)
                val_rmses.append(val_rmse)
        
        # Update history
        epoch_train_loss = np.mean(train_losses)
        epoch_val_loss = np.mean(val_losses)
        epoch_val_rmse = np.mean(val_rmses)
        
        history['train_loss'].append(epoch_train_loss)
        history['val_loss'].append(epoch_val_loss)
        history['val_rmse'].append(epoch_val_rmse)
        
        # Print progress
        print(f"Epoch {epoch}/{epochs} - Train loss: {epoch_train_loss:.4f} - Val RMSE: {epoch_val_rmse:.4f} - LR: {current_lr:.6f}")
        
        # Check early stopping (∆RMSE < 1%)
        if epoch_val_rmse < best_rmse * 0.99:  # Improvement of at least 1%
            best_rmse = epoch_val_rmse
            best_model_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
            print_progress(f"Epoch {epoch}: New best model with RMSE {best_rmse:.4f}", level=1)
            counter = 0
        else:
            counter += 1
        
        if counter >= patience:
            print_progress(f"Early stopping at epoch {epoch}", level=1)
            break
    
    # Restore best model
    model.load_state_dict(best_model_state)
    
    # Visualize learning curves
    plot_learning_curves(history, exp_name, fold)
    
    print_progress(f"Training of {exp_name} on fold {fold} completed. Best RMSE: {best_rmse:.4f}", is_end=True)
    
    # Save model
    torch.save(model.state_dict(), MODEL_DIR / f"{exp_name}_{fold}_model.pt")
    
    return model, history, best_rmse

def plot_learning_curves(history, exp_name, fold):
    """
    Generates learning curve visualizations during training
    
    Args:
        history: Dictionary with training history
        exp_name: Experiment name
        fold: Fold ID
    """
    curves_dir = IMAGE_DIR / "learning_curves"
    curves_dir.mkdir(exist_ok=True, parents=True)
    
    fig = plt.figure(figsize=(16, 12))
    gs = GridSpec(2, 2, figure=fig)
    
    # 1. Training and validation loss
    ax1 = fig.add_subplot(gs[0, 0])
    ax1.plot(history['train_loss'], label='Training', color='#3498db', linewidth=2)
    if 'val_loss' in history and len(history['val_loss']) > 0:
        ax1.plot(history['val_loss'], label='Validation', color='#e74c3c', linewidth=2)
    ax1.set_title('Loss during training', fontsize=14)
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Loss', fontsize=12)
    ax1.grid(alpha=0.3)
    ax1.legend(fontsize=12)
    
    # 2. Validation RMSE
    ax2 = fig.add_subplot(gs[0, 1])
    if 'val_rmse' in history and len(history['val_rmse']) > 0:
        ax2.plot(history['val_rmse'], color='#9b59b6', linewidth=2)
        min_rmse = min(history['val_rmse'])
        min_epoch = history['val_rmse'].index(min_rmse)
        ax2.scatter(min_epoch, min_rmse, c='red', s=100, zorder=10, label=f'Best: {min_rmse:.4f}')
    ax2.set_title('Validation RMSE', fontsize=14)
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('RMSE', fontsize=12)
    ax2.grid(alpha=0.3)
    ax2.legend(fontsize=12)
    
    # 3. Learning rate and Teacher Forcing
    ax3 = fig.add_subplot(gs[1, 0])
    if 'learning_rate' in history and len(history['learning_rate']) > 0:
        ax3.plot(history['learning_rate'], color='#2ecc71', linewidth=2)
        ax3.set_title('Learning Rate (OneCycleLR)', fontsize=14)
        ax3.set_xlabel('Epoch', fontsize=12)
        ax3.set_ylabel('Learning Rate', fontsize=12)
        ax3.set_yscale('log')
        ax3.grid(alpha=0.3)
    
    ax4 = fig.add_subplot(gs[1, 1])
    if 'teacher_forcing' in history and len(history['teacher_forcing']) > 0:
        ax4.plot(history['teacher_forcing'], color='#f39c12', linewidth=2)
        ax4.set_title('Teacher Forcing Ratio (0.7 → 0.3)', fontsize=14)
        ax4.set_xlabel('Epoch', fontsize=12)
        ax4.set_ylabel('Teacher Forcing', fontsize=12)
        ax4.set_ylim(0, 1)
        ax4.grid(alpha=0.3)
    
    plt.suptitle(f'{exp_name} - Fold {fold}', fontsize=16)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.savefig(curves_dir / f'{exp_name}_{fold}_learning_curves.png', dpi=100, bbox_inches='tight')
    plt.close(fig)
    
    print_progress(f"Learning curves saved at: {curves_dir / f'{exp_name}_{fold}_learning_curves.png'}", level=1)

# ▶️ Main experiment loop with learning curves and visualization
RESULTS = []
ALL_HISTORIES = {}
ALL_MODELS = {}

# Create folder for aggregated metrics
metrics_dir = MODEL_DIR / "metrics"
metrics_dir.mkdir(exist_ok=True, parents=True)

for exp_name, cfg in EXPERIMENTS.items():
    print_progress(f"Running experiment: {exp_name}", is_start=True)
    exp_histories = {}
    exp_models = {}
    exp_metrics = []
    
    for fold, val_year in FOLDS.items():
        print_progress(f"Processing fold {fold} (validation: {val_year})", level=1)
        
        # Build dataloaders
        train_loader, val_loader, in_dim = build_dataloaders(val_year, cfg['use_lags'])
        
        # Adjust dropout according to documentation (0.25 for F4-F5, 0.20 for others)
        dropout = 0.25 if fold in ['F4', 'F5'] else 0.20
        print_progress(f"Using dropout={dropout} for fold {fold}", level=2)
        print_progress(f"Input dimension: {in_dim}", level=2)
        
        # Create and train model with history tracking
        model = MODEL_FACTORY[cfg['model']](in_dim, dropout=dropout).to(DEVICE)
        model, history, best_rmse = train_with_history(
            model, train_loader, val_loader, 
            epochs=60, patience=20, 
            lr=1e-3, weight_decay=1e-4,
            fold=fold, exp_name=exp_name
        )
        
        # Save results
        RESULTS.append({
            'exp': exp_name,
            'fold': fold,
            'rmse': best_rmse
        })
        
        # Store model and history
        exp_histories[fold] = history
        exp_models[fold] = model
        
        # Generate prediction visualization if prepare_grid_data is implemented
        try:
            # Uncomment the following lines when prepare_grid_data is implemented
            # visualize_predictions(model, xr.open_dataset(FULL_NC), val_year, exp_name, fold)
            pass
        except Exception as e:
            print_progress(f"Error in visualization: {str(e)}", level=1)
    
    # Store histories and models
    ALL_HISTORIES[exp_name] = exp_histories
    ALL_MODELS[exp_name] = exp_models
    
    print_progress(f"Experiment {exp_name} completed", is_end=True)

# ▶️ Display results table
df = pd.DataFrame(RESULTS)
pivot_table = df.pivot(index='exp', columns='fold', values='rmse')
print_progress("RMSE results summary:", is_start=True)
display(pivot_table)

plt.title('RMSE comparison by experiment and fold', fontsize=14)
plt.xlabel('Experiment')
plt.ylabel('RMSE')
plt.xticks(rotation=45)
plt.grid(alpha=0.3)
plt.legend()
plt.tight_layout()
plt.savefig(IMAGE_DIR / "experiment_comparison.png", dpi=100)
plt.show()

def visualize_predictions(model, dataset, val_year, exp_name, fold, scalers=None):
    """
    Generates maps of predictions and MAPE errors for the 12 months of validation
    
    Args:
        model: Trained model
        dataset: Complete xarray dataset
        val_year: Validation year
        exp_name: Experiment name
        fold: Fold ID
        scalers: Tuple (sc_p, sc_x) of scalers to transform data
    """
    print_progress(f"Generating visualizations for {exp_name}, fold {fold}", is_start=True)
    
    # Prepare directory to save visualizations
    vis_dir = IMAGE_DIR / f"{exp_name}_{fold}_maps"
    vis_dir.mkdir(exist_ok=True, parents=True)
    
    # Get months from validation period
    months = pd.date_range(f"{val_year}-01-01", f"{val_year}-12-31", freq='MS')
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    # Extract coordinates
    lats = dataset.latitude.values
    lons = dataset.longitude.values
    
    # Create matrices to store results
    predictions = np.zeros((len(months), len(lats), len(lons)))
    true_values = np.zeros((len(months), len(lats), len(lons)))
    mape_values = np.zeros((len(months), len(lats), len(lons)))
    
    # Get time indices for validation
    val_times = dataset['time'].sel(time=slice(f"{val_year}-01-01", f"{val_year}-12-31")).values
    
    # Configure plots size
    plt.rcParams['figure.figsize'] = (20, 10)
    
    # Generate predictions for each grid point
    print_progress(f"Generating predictions", level=1)
    
    # This section depends on how your data is organized
    # Simplified example using a helper function
    input_tensor, target_tensor = prepare_grid_data(dataset, val_year, INPUT_WINDOW, HORIZON)
    
    # Make predictions
    with torch.no_grad():
        model.eval()
        preds = model(input_tensor.to(DEVICE)).cpu().numpy()
    
    # De-scale predictions if we have the scalers
    if scalers:
        sc_p, _ = scalers
        preds = sc_p.inverse_transform(preds.reshape(-1, HORIZON)).reshape(-1, len(lats), len(lons), HORIZON)
        # And rearrange axes to format (month, lat, lon)
        preds = np.moveaxis(preds, 3, 0)
    
    # We also need to extract real values and rearrange
    true_vals = target_tensor.numpy().reshape(-1, len(lats), len(lons), HORIZON)
    true_vals = np.moveaxis(true_vals, 3, 0)
    
    # Calculate MAPE
    for m in range(HORIZON):
        valid_mask = true_vals[m] > 0.1  # Avoid divisions by ~0
        mape_values[m, valid_mask] = np.abs((preds[m, valid_mask] - true_vals[m, valid_mask]) / true_vals[m, valid_mask]) * 100
    
    # Visualize maps for each month
    print_progress(f"Generating monthly maps", level=1)
    
    for m in range(HORIZON):
        fig = plt.figure(figsize=(18, 10))
        plt.suptitle(f"{exp_name} - {fold} - {month_names[m]} {val_year}", fontsize=16)
        
        # Prepare limits for colorbar
        vmin_pred = np.nanpercentile(true_vals, 1)
        vmax_pred = np.nanpercentile(true_vals, 99)
        vmin_mape = 0
        vmax_mape = min(100, np.nanpercentile(mape_values, 95))
        
        # Create grid for lat/lon
        lon2d, lat2d = np.meshgrid(lons, lats)
        
        # Prediction plot
        ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
        ax1.set_title(f"Predicted Precipitation (mm)")
        pcm = ax1.pcolormesh(lon2d, lat2d, preds[m], cmap='Blues', 
                           vmin=vmin_pred, vmax=vmax_pred, 
                           transform=ccrs.PlateCarree())
        ax1.coastlines(resolution='10m')
        ax1.add_feature(cfeature.BORDERS, linestyle=':')
        gl = ax1.gridlines(draw_labels=True, linewidth=0.5)
        gl.top_labels = False
        gl.right_labels = False
        plt.colorbar(pcm, ax=ax1, shrink=0.7, label='mm')
        
        # MAPE plot
        ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
        ax2.set_title(f"MAPE Error (%)")
        pcm2 = ax2.pcolormesh(lon2d, lat2d, mape_values[m], cmap='Reds', 
                             vmin=vmin_mape, vmax=vmax_mape, 
                             transform=ccrs.PlateCarree())
        ax2.coastlines(resolution='10m')
        ax2.add_feature(cfeature.BORDERS, linestyle=':')
        gl = ax2.gridlines(draw_labels=True, linewidth=0.5)
        gl.top_labels = False
        gl.right_labels = False
        plt.colorbar(pcm2, ax=ax2, shrink=0.7, label='%')
        
        # Save figure
        plt.tight_layout(rect=[0, 0, 1, 0.95])
        fig.savefig(vis_dir / f"map_{month_names[m]}.png", dpi=120, bbox_inches='tight')
        plt.close(fig)
    
    # Generate summary visualization (average)
    print_progress(f"Generating summary map", level=1)
    
    # Calculate averages
    avg_pred = np.nanmean(preds, axis=0)
    avg_true = np.nanmean(true_vals, axis=0)
    avg_mape = np.nanmean(mape_values, axis=0)
    
    # Summary plot
    fig = plt.figure(figsize=(18, 10))
    plt.suptitle(f"{exp_name} - {fold} - Annual Average {val_year}", fontsize=16)
    
    # Average prediction plot
    ax1 = plt.subplot(1, 2, 1, projection=ccrs.PlateCarree())
    ax1.set_title(f"Annual Mean Precipitation (mm)")
    pcm = ax1.pcolormesh(lon2d, lat2d, avg_pred, cmap='Blues', transform=ccrs.PlateCarree())
    ax1.coastlines(resolution='10m')
    ax1.add_feature(cfeature.BORDERS, linestyle=':')
    gl = ax1.gridlines(draw_labels=True, linewidth=0.5)
    gl.top_labels = False
    gl.right_labels = False
    plt.colorbar(pcm, ax=ax1, shrink=0.7, label='mm')
    
    # Average MAPE plot
    ax2 = plt.subplot(1, 2, 2, projection=ccrs.PlateCarree())
    ax2.set_title(f"Average MAPE (%)")
    pcm2 = ax2.pcolormesh(lon2d, lat2d, avg_mape, cmap='Reds', 
                         vmin=0, vmax=min(100, np.nanpercentile(avg_mape, 95)), 
                         transform=ccrs.PlateCarree())
    ax2.coastlines(resolution='10m')
    ax2.add_feature(cfeature.BORDERS, linestyle=':')
    gl = ax2.gridlines(draw_labels=True, linewidth=0.5)
    gl.top_labels = False
    gl.right_labels = False
    plt.colorbar(pcm2, ax=ax2, shrink=0.7, label='%')
    
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    fig.savefig(vis_dir / f"map_annual_summary.png", dpi=120, bbox_inches='tight')
    plt.close(fig)
    
    # RMSE by horizon plot (1-12)
    rmse_by_horizon = [np.sqrt(np.nanmean((preds[h] - true_vals[h])**2)) for h in range(HORIZON)]
    
    fig = plt.figure(figsize=(10, 6))
    plt.plot(range(1, HORIZON+1), rmse_by_horizon, marker='o', linewidth=2)
    plt.title(f"{exp_name} - {fold} - RMSE by Horizon", fontsize=14)
    plt.xlabel('Prediction Horizon (months)', fontsize=12)
    plt.ylabel('RMSE', fontsize=12)
    plt.grid(alpha=0.3)
    plt.xticks(range(1, HORIZON+1))
    plt.tight_layout()
    fig.savefig(vis_dir / f"rmse_by_horizon.png", dpi=120)
    plt.close(fig)
    
    print_progress(f"Visualizations saved in {vis_dir}", is_end=True)
    return preds, true_vals, mape_values

# Helper function to prepare data in grid format
def prepare_grid_data(dataset, val_year, input_window, horizon):
    """
    Prepares input and target data for grid predictions
    
    This function is a placeholder - you'll need to implement it according to
    your specific data structure
    """
    print_progress("This function needs specific implementation for the dataset!", level=2)
    # Placeholder - returns empty tensors
    return torch.zeros((1, input_window, 10)), torch.zeros((1, horizon))

# ▶️ Performance and GPU optimizations for training
import torch.cuda.amp as amp  # For mixed precision

# PyTorch memory and performance optimizations
torch.backends.cudnn.benchmark = True  # Optimize repetitive operations
torch.backends.cudnn.enabled = True    # Ensure cuDNN is enabled
torch.backends.cuda.matmul.allow_tf32 = True  # Allow TF32 on Ampere GPUs
torch.backends.cudnn.allow_tf32 = True        # Allow TF32 in convolution operations

# Optimized parameters for better GPU utilization
BATCH_SIZE_FAST = 128          # Larger batches for better GPU utilization
EPOCHS_FAST =  10               # Quick training for exploration
TRANSFER_MODE = 'non_blocking'  # Asynchronous CPU-GPU transfer

# Function to monitor and free GPU memory

def gpu_monitor(reset=False):
    """Monitors GPU memory usage and optionally frees it"""
    if torch.cuda.is_available():
        mem_alloc = torch.cuda.memory_allocated()/1e9  # GB
        mem_reserved = torch.cuda.memory_reserved()/1e9  # GB
        print(f"🧠 GPU: {mem_alloc:.2f} GB alloc | {mem_reserved:.2f} GB reserved")
        if reset:
            print("🧹 Freeing GPU memory...")
            torch.cuda.empty_cache()
            print(f"   After: {torch.cuda.memory_allocated()/1e9:.2f} GB")
    else:
        print("❌ GPU not available")

# ▶️ Memory management functions
def aggressive_memory_cleanup():
    """Aggressively frees memory resources, especially GPU memory"""
    # Empty CUDA cache if available
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # Force garbage collection
    import gc
    gc.collect()
    
    # Close any matplotlib figures that may be open
    try:
        plt.close('all')
    except:
        pass
    
    # Clean PyTorch memory
    for obj in gc.get_objects():
        try:
            if torch.is_tensor(obj) and obj.is_cuda and not obj.is_grad:
                del obj
        except:
            pass

# ▶️ Additional optimizations to speed up training

def optimize_training_pipeline():
    """Implements additional optimizations to speed up training"""
    enhanced_logger("Applying additional optimizations to accelerate training...", is_start=True)
    
    # 1. Data loading optimizations
    # Pre-load and cache in memory for small datasets
    if ENV_INFO['memory']['available_gb'] > 20:  # If enough RAM
        enhanced_logger("Enabling data caching in memory", level=1)
        torch.utils.data.dataloader.default_collate = lambda x: x  # Avoid unnecessary reconstruction
    
    # 2. JIT compilation optimizations for critical functions
    if hasattr(torch, 'compile'):  # PyTorch 2.0+
        enhanced_logger("Enabling JIT compilation for critical functions (PyTorch 2.0+)", level=1)
        # Functions will be compiled when models are defined
    else:
        enhanced_logger("PyTorch compile not available (requires PyTorch 2.0+)", level=1)
    
    # 3. Optimize GPU-specific parameters
    if DEVICE.type == 'cuda':
        # Reserve cache memory to avoid fragmentation
        if ENV_INFO['torch']['cuda_available'] and torch.cuda.is_available():
            enhanced_logger("Optimizing GPU memory...", level=1)
            # Set memory limits based on detected GPU
            if any('T4' in gpu['name'] for gpu in ENV_INFO['gpu']):
                # Configuration for Google Colab T4 (16GB)
                torch.cuda.set_per_process_memory_fraction(0.85)  # Use 85% of memory
                enhanced_logger("Optimized configuration for T4 GPU", level=2)
                torch.cuda.set_per_process_memory_fraction(0.85)  # Use 85% of memory
                enhanced_logger("Optimized configuration for T4 GPU", level=2)
            elif any('K80' in gpu['name'] for gpu in ENV_INFO['gpu']):
                # Configuration for Google Colab K80 (12GB)
                torch.cuda.set_per_process_memory_fraction(0.8)  # Use 80% of memory
                enhanced_logger("Optimized configuration for K80 GPU", level=2)
            elif sum(gpu['memory_gb'] for gpu in ENV_INFO['gpu']) > 24:
                # High-memory GPU (>24GB)
                batch_mult = 2.0
                enhanced_logger(f"High memory GPU detected - batch multiplier x{batch_mult}", level=2)
            else:
                # Generic configuration
                torch.cuda.set_per_process_memory_fraction(0.75)  # Use 75% for buffer
    
    # 4. Adjust data transfer strategy based on hardware
    if ENV_INFO['torch']['cuda_available']:
        if ENV_INFO['cpu']['cores_physical'] > 12:
            # For powerful CPUs, use more workers for transfer
            global NUM_WORKERS
            NUM_WORKERS = min(8, ENV_INFO['cpu']['cores_logical'] // 2)
            enhanced_logger(f"Powerful CPU detected - increasing workers to {NUM_WORKERS}", level=1)
    
    enhanced_logger("Optimizations successfully applied", is_end=True)
    return True

# Call this function before starting training
optimize_training_pipeline()

# Add JIT compilation for GRU (only if PyTorch >= 2.0)
if hasattr(torch, 'compile'):
    class CompiledGRUEncoderDecoder(GRUEncoderDecoder):
        def __init__(self, *args, **kwargs):
            super().__init__(*args, **kwargs)
            # Compile critical parts to optimize performance
            self.forward_compiled = torch.compile(super().forward)
        
        def forward(self, x, teacher_forcing_ratio=0.5, y=None):
            return self.forward_compiled(x, teacher_forcing_ratio, y)
    
    # Replace in MODEL_FACTORY
    MODEL_FACTORY['gru_ed'] = CompiledGRUEncoderDecoder
    enhanced_logger("GRU models compiled with torch.compile for faster speed", level=1)

# Modify build_dataloaders to implement data caching
def fast_build_dataloaders(val_year, use_lags, batch_size=BATCH_SIZE):
    """Optimized version of the dataloader builder with caching and efficient sampling"""
    # Use global cache to avoid reloading datasets
    global _dataset_cache
    if not '_dataset_cache' in globals():
        _dataset_cache = {}
    
    cache_key = f"{val_year}_{use_lags}"
    if cache_key in _dataset_cache:
        enhanced_logger(f"Using cached dataset for {val_year}", level=1)
        train_loader, val_loader, real_feature_dim = _dataset_cache[cache_key]
        
        # Update only batch size if different
        if train_loader.batch_size != batch_size:
            train_loader = DataLoader(
                train_loader.dataset,
                batch_size=batch_size,
                shuffle=True,
                num_workers=NUM_WORKERS,
                pin_memory=True,
                persistent_workers=NUM_WORKERS > 0,
                prefetch_factor=2 if NUM_WORKERS > 0 else None
            )
            
            val_loader = DataLoader(
                val_loader.dataset,
                batch_size=batch_size,
                shuffle=False,
                num_workers=NUM_WORKERS,
                pin_memory=True,
                persistent_workers=NUM_WORKERS > 0,
                prefetch_factor=2 if NUM_WORKERS > 0 else None
            )
        
        return train_loader, val_loader, real_feature_dim
    
    # If not in cache, build normally
    train_loader, val_loader, real_feature_dim = build_dataloaders(val_year, use_lags, batch_size)
    
    # Save in cache
    _dataset_cache[cache_key] = (train_loader, val_loader, real_feature_dim)
    
    return train_loader, val_loader, real_feature_dim

# Modify run_experiments to use these optimizations
def run_experiments_fast(fast_mode=False, transfer_learning=False):
    """Optimized version for fast execution of experiments"""
    # Apply optimizations
    optimize_training_pipeline()
    
    # Use partial function to avoid modifying all the code
    import functools
    original_build = build_dataloaders
    build_dataloaders = fast_build_dataloaders
    
    try:
        # Run with optimized pipeline
        return run_experiments(fast_mode, transfer_learning)
    finally:
        # Restore original function
        build_dataloaders = original_build

# Define the missing run_experiments function that was referenced but not implemented
def run_experiments(fast_mode=False, transfer_learning=False):
    """
    Runs the main pipeline experiments
    
    Args:
        fast_mode: If True, uses fewer epochs and larger batch size for quick tests
        transfer_learning: If True, initializes each model with the best from the previous fold
    """
    results = []
    all_histories = {}
    all_models = {}
    
    # Create folder for aggregated metrics if it doesn't exist
    metrics_dir = MODEL_DIR / "metrics"
    metrics_dir.mkdir(exist_ok=True, parents=True)
    
    # Configure epochs based on mode
    epochs = 20 if fast_mode else 60
    batch_size = BATCH_SIZE_FAST if fast_mode else BATCH_SIZE
    
    for exp_name, cfg in EXPERIMENTS.items():
        print_progress(f"Running experiment: {exp_name}", is_start=True)
        exp_histories = {}
        exp_models = {}
        
        prev_model = None  # For transfer learning
        
        for fold, val_year in FOLDS.items():
            print_progress(f"Processing fold {fold} (validation: {val_year})", level=1)
            
            # Build dataloaders
            train_loader, val_loader, in_dim = build_dataloaders(val_year, cfg['use_lags'], batch_size)
            
            # Adjust dropout according to documentation
            dropout = 0.25 if fold in ['F4', 'F5'] else 0.20
            print_progress(f"Using dropout={dropout} for fold {fold}", level=2)
            print_progress(f"Input dimension: {in_dim}", level=2)
            
            # Create model
            model = MODEL_FACTORY[cfg['model']](in_dim, dropout=dropout).to(DEVICE)
            
            # Apply transfer learning if enabled and there's a previous model
            if transfer_learning and prev_model is not None:
                print_progress(f"Applying transfer learning from previous fold", level=2)
                # Copy weights from previous model that match in size
                with torch.no_grad():
                    for name, param in model.named_parameters():
                        if name in prev_model.state_dict() and param.size() == prev_model.state_dict()[name].size():
                            param.copy_(prev_model.state_dict()[name])
            
            # Train model
            model, history, best_rmse = train_with_history(
                model, train_loader, val_loader,
                epochs=epochs, patience=20 if not fast_mode else 10,
                lr=1e-3, weight_decay=1e-4,
                fold=fold, exp_name=exp_name
            )
            
            # Save results
            results.append({
                'exp': exp_name,
                'fold': fold,
                'rmse': best_rmse
            })
            
            # Store model and history
            exp_histories[fold] = history
            exp_models[fold] = model
            
            # Save as previous model for transfer learning
            prev_model = model
            
            # Generate visualization if prepare_grid_data is implemented
            try:
                # Visualize predictions if function is implemented
                # visualize_predictions(model, xr.open_dataset(FULL_NC), val_year, exp_name, fold)
                pass
            except Exception as e:
                print_progress(f"Error in visualization: {str(e)}", level=1)
        
        # Store histories and models
        all_histories[exp_name] = exp_histories
        all_models[exp_name] = exp_models
        
        print_progress(f"Experiment {exp_name} completed", is_end=True)
    
    return results, all_histories, all_models